In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torchstain --no-index --find-links=file:///kaggle/input/torchstain
import torchstain
print("torchstain package successfully imported!")

Looking in links: file:///kaggle/input/torchstain
Processing /kaggle/input/torchstain/torchstain-1.3.0-py3-none-any.whl
torchstain package successfully imported!


In [3]:
from tensorflow.keras.models import load_model
#CNN_model = load_model('/kaggle/input/model12/CNN_model.h5')
#VGG_model = load_model('/kaggle/input/model21/vgg_model.h5')
VGG_model_16 = load_model('/kaggle/input/cute-16/vgg_model_16.h5')
import joblib
target = joblib.load('/kaggle/input/preprocessing-done/target.joblib')

In [4]:
import pandas as pd
test_csv_path = "/kaggle/input/UBC-OCEAN/test.csv"
test_df = pd.read_csv(test_csv_path)
test_df

,image_id,image_width,image_height
0,41,28469,16987


In [5]:
test_df['is_tma']=((test_df['image_width'] < 5000) & (test_df['image_height'] < 5000))
test_df

,image_id,image_width,image_height,is_tma
0,41,28469,16987,False


In [6]:
test_df['is_tma']=test_df['is_tma'].astype("int8")
test_df['image_id']=test_df['image_id'].astype("int32")
test_df['image_width']=test_df['image_width'].astype("int32")
test_df['image_height']=test_df['image_height'].astype("int32")
test_df

,image_id,image_width,image_height,is_tma
0,41,28469,16987,0


In [7]:
from PIL import Image 
import numpy as np
Image.MAX_IMAGE_PIXELS = 10000000000
# Define patch size and overlap (if needed)  
patch_size = (512,512)# 提取图块大小 Adjust this according to your requirements
overlap = 20  # 相邻图块之间的重叠 Adjust this if you want overlapping patches
resize_size = (256, 256)  # New size for the resized image
def slice(test_df):
    image_data = []
    img_ids=[] #imageid方便后面majority voting
    empty_img=0
    for img_id,  tma in zip(test_df['image_id'], test_df['is_tma']):
        #print(img_id, label,  tma)
        if tma==0:
            img_name = str(img_id)+"_thumbnail.png"
            large_image = Image.open("/kaggle/input/UBC-OCEAN/test_thumbnails/"+img_name)
            for y in range(0, large_image.height, patch_size[0] - overlap): # (0,2523,192)
                for x in range(0, large_image.width, patch_size[1] - overlap):  # (0,3000,192)  224-32=192
                    patch = large_image.crop((x, y, x+patch_size[1], y+patch_size[0]))
                    resized_patch = patch.resize(resize_size)
                    image = np.array(resized_patch)
                    #image = np.array(patch)
                    if np.sum(image)==0:
                        empty_img+=1
                    elif (np.sum(image[0:,0:50])==0) or (np.sum(image[0:,50:])==0) or (np.sum(image[0:,100:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:50,0:])==0) or (np.sum(image[50:,0:])==0) or (np.sum(image[100:,0:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:50,0:50])==0) or (np.sum(image[0:50,50:])==0):
                        empty_img+=1
                    elif (np.sum(image[50:100,0:50])==0) or (np.sum(image[50:100,50:])==0):
                        empty_img+=1
                    elif (np.sum(image[50:,0:100])==0) or (np.sum(image[80:,80:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:50,75:])==0) or (np.sum(image[50:,75:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:40,80:])==0) or (np.sum(image[80:,80:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:35,0:])==0) or (np.sum(image[80:,0:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:25,0:50])==0) or (np.sum(image[0:25,100:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:20,0:25])==0) or (np.sum(image[0:20,25:50])==0) or (np.sum(image[0:20,50:80])==0) or (np.sum(image[0:20,90:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:25,0:25])==0) or (np.sum(image[0:25,100:])==0) or (np.sum(image[100:,0:25])==0) or (np.sum(image[100:,100:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:15,0:])==0) or (np.sum(image[115:,0:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:,0:15])==0) or (np.sum(image[0:,115:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:20,0:20])==0) or (np.sum(image[0:20,110:])==0) or (np.sum(image[110:,0:20])==0) or (np.sum(image[110:,110:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:10,0:10])==0) or (np.sum(image[0:10,115:])==0) or (np.sum(image[0:10,40:60])==0) or (np.sum(image[0:10,80:100])==0):
                        empty_img+=1
                    elif (np.sum(image[40:50,0:10])==0) or (np.sum(image[110:,80:100])==0) or (np.sum(image[70:85,70:90])==0) or (np.sum(image[50:60,110:])==0):
                        empty_img+=1

                    else:
                        image_data.append(image)
                        img_ids.append(img_id)
        elif tma==1:
            img_name = str(img_id)+".png"
            large_image = Image.open("/kaggle/input/UBC-OCEAN/test_images/"+img_name)
            for y in range(0, large_image.height, patch_size[0] - overlap): # (0,2523,192)
                for x in range(0, large_image.width, patch_size[1] - overlap):  # (0,3000,192)  224-32=192
                    patch = large_image.crop((x, y, x+patch_size[1], y+patch_size[0]))
                    resized_patch = patch.resize(resize_size)
                    image = np.array(resized_patch)
                    if np.sum(image)==0:
                        empty_img+=1
                    elif (np.sum(image[0:,0:50])==0) or (np.sum(image[0:,50:])==0) or (np.sum(image[0:,100:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:50,0:])==0) or (np.sum(image[50:,0:])==0) or (np.sum(image[100:,0:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:50,0:50])==0) or (np.sum(image[0:50,50:])==0):
                        empty_img+=1
                    elif (np.sum(image[50:100,0:50])==0) or (np.sum(image[50:100,50:])==0):
                        empty_img+=1
                    elif (np.sum(image[50:,0:100])==0) or (np.sum(image[80:,80:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:50,75:])==0) or (np.sum(image[50:,75:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:40,80:])==0) or (np.sum(image[80:,80:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:35,0:])==0) or (np.sum(image[80:,0:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:25,0:50])==0) or (np.sum(image[0:25,100:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:20,0:25])==0) or (np.sum(image[0:20,25:50])==0) or (np.sum(image[0:20,50:80])==0) or (np.sum(image[0:20,90:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:25,0:25])==0) or (np.sum(image[0:25,100:])==0) or (np.sum(image[100:,0:25])==0) or (np.sum(image[100:,100:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:15,0:])==0) or (np.sum(image[115:,0:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:,0:15])==0) or (np.sum(image[0:,115:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:20,0:20])==0) or (np.sum(image[0:20,110:])==0) or (np.sum(image[110:,0:20])==0) or (np.sum(image[110:,110:])==0):
                        empty_img+=1
                    elif (np.sum(image[0:10,0:10])==0) or (np.sum(image[0:10,115:])==0) or (np.sum(image[0:10,40:60])==0) or (np.sum(image[0:10,80:100])==0):
                        empty_img+=1
                    elif (np.sum(image[40:50,0:10])==0) or (np.sum(image[110:,80:100])==0) or (np.sum(image[70:85,70:90])==0) or (np.sum(image[50:60,110:])==0):
                        empty_img+=1


                    else:
                        image_data.append(image)
                        img_ids.append(img_id)
    img_ids = np.array(img_ids)
    return image_data,img_ids

In [8]:
import cv2
import torchstain
import torch
from torchvision import transforms
import time
def transform(image_data):
    transformed_image = []

    for i in range(0, len(image_data)):
        #print(i)
        to_transform = cv2.cvtColor(image_data[i], cv2.COLOR_BGR2RGB)

        T = transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x * 255)
        ])

        normalizer = torchstain.normalizers.MacenkoNormalizer(backend='torch')

        try:
            normalizer.fit(T(target))
            t_to_transform = T(to_transform)
            norm, H, E = normalizer.normalize(I=t_to_transform, stains=True)
            transformed_image.append(norm)
        except (IndexError, torch.linalg.LinAlgError) as e:
            print(f"Error processing image at index {i}: {e}")
            # If an error occurs, append the original image
            transformed_image.append(to_transform)
    return transformed_image

In [9]:
def normalize(transformed_image):
    normalized_image=[]
    for i in range(0,len(transformed_image) ):

        # Your image data
        image = np.array(transformed_image[i])  # convert torch.Size([256, 256, 3]) to np array

        # Define ImageNet mean and standard deviation values for each channel
        imagenet_mean = np.array([0.485, 0.456, 0.406])
        imagenet_std = np.array([0.229, 0.224, 0.225])

        # Step 1: Convert to float32
        image_data_float = image.astype(np.float16)

        # Step 2: Normalize to [0, 1]
        image_data_normalized2 = image_data_float / 255.0

        # Step 3: Subtract mean
        image_data_normalized2 -= imagenet_mean

        # Step 4: Divide by standard deviation
        image_data_normalized2 /= imagenet_std

        # Clip values to be within the range [-1, 1]
        image_data_normalized2 = np.clip(image_data_normalized2, -1, 1)
        normalized_image.append(image_data_normalized2)
    return normalized_image

In [10]:
from collections import Counter

def find_majority_label(image_ids, labels):
    id_label_map = {}  # 创建一个空的字典来存储image_id和对应的label

    # 遍历image_ids和labels列表
    for image_id, label in zip(image_ids, labels):
        if image_id in id_label_map:
            id_label_map[image_id].append(label)
        else:
            id_label_map[image_id] = [label]

    majority_labels = {}  # 创建一个空的字典来存储每个唯一image_id的多数投票label

    # 对字典中的每个唯一image_id的label列表进行多数投票
    for image_id, label_list in id_label_map.items():
        label_counts = Counter(label_list)
        majority_label = label_counts.most_common(1)[0][0]
        majority_labels[image_id] = majority_label

    return majority_labels

In [11]:
import pandas as pd
from collections import Counter
import numpy as np
import tensorflow as tf

tf.data.experimental.enable_debug_mode()
tf.config.run_functions_eagerly(True)
predicted_labels = []
for i in range(len(test_df)):
    image_data, img_ids = slice(test_df.iloc[[i]])
    if image_data==[]:
        predicted_label='HGSC'
    else:
        transformed_image = transform(image_data)
        normalized_image = normalize(transformed_image)
        del transformed_image
        x = np.array(normalized_image) 
        del normalized_image
        predictions = VGG_model_16.predict(x)
        del x
        y_pred = np.argmax(predictions, axis=1)
        del predictions
        y_votingpred = find_majority_label(img_ids, y_pred)
        unique_img_ids = id  
        label = list(y_votingpred.values())[0]
        shortform_to_label = {0: 'HGSC', 1: 'LGSC', 2: 'EC', 3: 'CC', 4: 'MC', 5: 'Other'}
        predicted_label = shortform_to_label[label]
    #print(predicted_label)
    del image_data,img_ids
    predicted_labels.append(predicted_label)
predicted_labels

1/1 [==============================] - 3s 3s/step


['HGSC']

In [12]:
'''import pandas as pd
from collections import Counter
import numpy as np
import tensorflow as tf

tf.data.experimental.enable_debug_mode()
tf.config.run_functions_eagerly(True)
predicted_labels = []
for i in range(len(test_df)):
    image_data, img_ids = slice(test_df.iloc[[i]])
    if image_data==[]:
        predicted_label='HGSC'
    else:
        transformed_image = transform(image_data)
        normalized_image = normalize(transformed_image)
        x = np.array(normalized_image) 
        predictions = VGG_model_16.predict(x)
        y_pred = np.argmax(predictions, axis=1)
        y_votingpred = find_majority_label(img_ids, y_pred)
        unique_img_ids = id  
        label = list(y_votingpred.values())[0]
        shortform_to_label = {0: 'HGSC', 1: 'LGSC', 2: 'EC', 3: 'CC', 4: 'MC', 5: 'Other'}
        predicted_label = shortform_to_label[label]
        del transformed_image,normalized_image,predictions,x
    #print(predicted_label)
    del image_data,img_ids
    predicted_labels.append(predicted_label)
predicted_labels'''

"import pandas as pd\nfrom collections import Counter\nimport numpy as np\nimport tensorflow as tf\n\ntf.data.experimental.enable_debug_mode()\ntf.config.run_functions_eagerly(True)\npredicted_labels = []\nfor i in range(len(test_df)):\n    image_data, img_ids = slice(test_df.iloc[[i]])\n    if image_data==[]:\n        predicted_label='HGSC'\n    else:\n        transformed_image = transform(image_data)\n        normalized_image = normalize(transformed_image)\n        x = np.array(normalized_image) \n        predictions = VGG_model_16.predict(x)\n        y_pred = np.argmax(predictions, axis=1)\n        y_votingpred = find_majority_label(img_ids, y_pred)\n        unique_img_ids = id  \n        label = list(y_votingpred.values())[0]\n        shortform_to_label = {0: 'HGSC', 1: 'LGSC', 2: 'EC', 3: 'CC', 4: 'MC', 5: 'Other'}\n        predicted_label = shortform_to_label[label]\n        del transformed_image,normalized_image,predictions,x\n    #print(predicted_label)\n    del image_data,img_

In [13]:
submission_data = {'image_id': test_df['image_id'], 'label': predicted_labels}
submission_df = pd.DataFrame(submission_data)
submission_df

,image_id,label
0,41,HGSC


In [14]:
submission_df.to_csv('submission.csv', index=False) 